## On this notebook we extract Censo 2010 individual data from their files.


In [118]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


/tmp/ipykernel_184550/3391910190.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [119]:
import os

if not os.path.exists('./../../samplerCensoARG/'):
    !git clone https://github.com/matuteiglesias/samplerCensoARG.git ./../../

In [120]:
# !python ./../../samplerCensoARG/samplear.py -dbp '/media/miglesia/Elements/suite/ext_CPV2010_basico_radio_pub' -f 0.01 -y 2005 2006 -n ARG

In [121]:
import pandas as pd
# pd.read_csv('/home/miglesia/repositories/indice-pobreza-ExactasUBA/notebooks/../data/info/AGLO_rk/')

## Cargar info accesoria (Regiones, aglomerados, etc)

In [122]:
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')

radio_AGLO = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/Aglomerados-EPH-INDEC/main/result/radios_aglo_EPH.csv')
radio_AGLO['radio'] = radio_AGLO.COD_2010.str.replace('XX', '99').astype(int)
radio_AGLO['AGLOMERADO'] = radio_AGLO.eph_codagl
radio_AGLO['NOMAGLO'] = radio_AGLO.eph_aglome

radio_ref = radio_ref.drop(['AGLOMERADO'], axis = 1).merge(radio_AGLO[['radio','AGLOMERADO', 'NOMAGLO']], how = 'left')
radio_ref['AGLOMERADO'] = radio_ref['AGLOMERADO'].fillna(0).astype(int)

# radio_ref[['PROV','NOMPROV','DPTO', 'NOMDPTO']].drop_duplicates().to_csv('./../data/DPTO_PROV.csv', index = False)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)

In [123]:

AGLO_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/AGLO_rk')
rk_table = AGLO_rk.set_index(['ANO4', 'AGLOMERADO']).unstack()
AGLO_rk_filled = rk_table.fillna(rk_table.mean()).stack().reset_index()
AGLO_rk = AGLO_rk_filled

Reg_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/Reg_rk')
Reg_rk.head()


,ANO4,Region,Reg_rk
0,2003,cuyo,0.500
1,2003,gran_buenos_aires,0.833
2,2003,patagonia,1.000
3,2003,pampeana,0.667
4,2003,noroeste,0.167


## Loop sampleo de censo

In [124]:
import numpy as np

In [125]:
## CAMBIADOR DE CONVENCIONES NOMBRE REGION
# # File path
# # file_path = './../../encuestador-de-hogares/data/info/Reg_rk'
# file_path = './../data/info/DPTO_PROV_Region.csv'


# # Read the old copy
# Reg_rk = pd.read_csv(file_path)

# # Define the mapping dictionary
regiones = {
    'Gran Buenos Aires': 'gran_buenos_aires',
    'Pampeana': 'pampeana',
    'Noroeste': 'noroeste',
    'Noreste': 'noreste',
    'Patagónica': 'patagonia',
    'Cuyo': 'cuyo'
}

# # Update region names using the mapping dictionary
# Reg_rk['Region'] = Reg_rk['Region'].map(regiones)

# # Save the updated DataFrame back to the same file
# Reg_rk.to_csv(file_path, index=False)

# # Print a message to confirm the update
# print("Region names have been updated and saved successfully!")

In [126]:
frac = 0.05 ## Frac needs to be the fraction used in the sampling (eg. -f 0.01 needs frac = 0.01)
startyr = 2019
endyr = 2023

for yr in [str(s) for s in range(startyr, endyr)]:

    ## EXTRACCION
    ##############
    print(yr)
    input = pd.read_csv('./../../samplerCensoARG/data/censo_samples/table_f'+str(frac)+'_'+yr+'_ARG.csv')
    table = input.copy()
    table['ANO4'] = int(yr)
    

    ## TRANSFORMACION
    ##################
    # Adaptamos las categorias de respuestas para que iguales las de la EPH
    ## VIVIENDA
    table['V01'] = table['V01'].map({1:1, 2:6, 3:6, 4:2, 5:3, 6:4, 7:5, 8:6})
    ## HOGAR
    table['H06'] = table['H06'].map({1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:9})
    table['H09'] = table['H09'].map({1:1, 2:2, 3:3, 4:4, 5:4, 6:4})
    table['H16'] = table['H16'].clip(0, 9)
    table['H14'] = table['H14'].map({1:1, 2:4, 3:2, 4:2, 5:4, 6:3, 7:4, 8:9})
    table['H13'] = table['H13'].map({1:1, 2:2, 4:0})
    table['IX_TOT'] = table['TOTPERS']
    # PERSONA
    table['P07'] = table['P07'].map({1:1, 2:2, 0:2})

    ## Agregar Region
    table = table.merge(dpto_region[['DPTO', 'Region']])

    ## ID DE PERSONAS SIMULADAS:
    # Semilla ids
    file_size = os.path.getsize('./../../samplerCensoARG/data/censo_samples/table_f'+str(frac)+'_'+yr+'_ARG.csv')
    np.random.seed(file_size)

    ## Aplicar IDs
    # Generate a unique random number for each row in the DataFrame
    n_digits = 8; n_rows = len(table)
    random_numbers = np.random.randint(10**(n_digits - 1), 10**n_digits, n_rows)
    # Extract the last two digits of the year
    last_two_digits_year = table['ANO4'].apply(lambda x: int(str(x)[-2:]))
    
    # unique ID
    table.insert(0, 'ID', random_numbers * 100 + last_two_digits_year)

    ## Agregar ranking de Region y Aglo
    print(table.shape)
    table = table.merge(AGLO_rk[['AGLOMERADO', 'ANO4', 'AGLO_rk']]).merge(Reg_rk[['Region', 'ANO4', 'Reg_rk']])
    print(table.shape)
    

    # CARGA
    #############
    table.to_csv('/media/matias/Elements/suite/poblaciones/table_f'+str(frac)+'_'+yr+'_ARG.csv', index = False)  # Copias en carpeta yr_samples, en nuestra carpeta de indice de pobreza
    

2019
(2216657, 39)
(2216657, 41)


Define Source and Destination: Identify the source of your data (CSV files in this case) and the destination (e.g., a relational database).

Create Tables and Relationships: Design the database schema to match the transformed data structure, including tables, columns, and relationships.

Build ETL Pipeline: Develop a pipeline using ETL tools or scripting to extract, transform, and load the data as described above.

Validation and Error Handling: Include validation steps to ensure data quality and error handling to deal with any issues that arise during the transformation.

Scheduling and Monitoring: If the process needs to be run regularly, set up scheduling and monitoring to ensure that it runs smoothly and that any issues are quickly identified and addressed.

Documentation and Compliance: Document the entire process, including the transformations and mappings, especially if there are regulatory compliance requirements related to the data.